This tutorial shows how to reconstruct L4A aboveground biomass density (AGBD) estimates using L2A relative height (RH) metrics. We will use a GEDI L4A file `GEDI04_A_2020207182449_O09168_03_T03028_02_002_01_V002.h5` and corresponding GEDI L2A file `GEDI02_A_2020207182449_O09168_03_T03028_02_003_01_V002.h5` for this purpose.

In [1]:
# import all the modules
from os import path
import h5py
import numpy as np

## L4A file and prediction stratum

First, open the L4A file:

In [2]:
# GEDI L4a file
l4a = 'GEDI04_A_2020207182449_O09168_03_T03028_02_002_01_V002.h5'
l4af = path.join('full_orbits', l4a)
# read the L4A file
hf_l4a = h5py.File(l4af, 'r')

This tutorial will use data for a single shot (shot number `91680600300633870`) from the beam `BEAM0110`. Let's find out its predict_stratum and selected_algorithm.

In [3]:
shot_number_l4a = hf_l4a['BEAM0110']['shot_number'][158560:158561][0]
predict_stratum_l4a = hf_l4a['BEAM0110']['predict_stratum'][158560:158561][0]
selected_algorithm_l4a = hf_l4a['BEAM0110']['selected_algorithm'][158560:158561][0]

print_s = f"""For GEDI shot number {shot_number_l4a}, \
the predict stratum is {predict_stratum_l4a.decode('UTF-8')}, \
and the selected algorithm group is {selected_algorithm_l4a}."""
print(print_s)

For GEDI shot number 91680600300633870, the predict stratum is EBT_SAs, and the selected algorithm group is 2.


## L4A model parameters
Let's print the model_data information for the `EBT_SAs` stratum.

In [4]:
model_data_l4a = hf_l4a['ANCILLARY']['model_data']
model_stratum_l4a = model_data_l4a[12] # EBT_SAs
for v in model_stratum_l4a.dtype.names:
    print(v, " : ", model_stratum_l4a[v])

predict_stratum  :  b'EBT_SAs'
model_group  :  4
model_name  :  b'EBT_coarse'
model_id  :  1
x_transform  :  b'sqrt'
y_transform  :  b'sqrt'
bias_correction_name  :  b'Snowdon'
fit_stratum  :  b'EBT'
rh_index  :  [50 98  0  0  0  0  0  0]
predictor_id  :  [1 2 0 0 0 0 0 0]
predictor_max_value  :  [12.66807  13.052969  0.        0.        0.        0.        0.
  0.      ]
vcov  :  [[ 1.6210171  -0.10003732 -0.047196    0.          0.        ]
 [-0.10003732  0.05625523 -0.04469026  0.          0.        ]
 [-0.047196   -0.04469026  0.04664591  0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.        ]]
par  :  [-104.9654541     6.80217409    3.95531225    0.            0.        ]
rse  :  3.9209127
dof  :  4811
response_max_value  :  1578.0
bias_correction_value  :  1.1133657
npar  :  3


A square-root transformation is applied to both predictor and response variables as indicated by `x_transform` and `y_transform` above. Take a note of the `bias_correction_value` and `bias_correction_name` above. We will use the `bias_correction_value` later to back-transform AGBD values. Now, read all the model parameters and print the AGBD model.

In [5]:
intercept = model_stratum_l4a['par'][0] # intercept
coeff_1 = model_stratum_l4a['par'][1] # first coeff
coeff_2 = model_stratum_l4a['par'][2] # first coeff
predict_1 = model_stratum_l4a['rh_index'][0] # predictor 1
predict_2 = model_stratum_l4a['rh_index'][1] # predictor 1
# print
print_s = f"""The predict stratum {predict_stratum_l4a.decode('UTF-8')} uses following model:
AGBD = {intercept} + {coeff_1} x RH_{predict_1} + {coeff_2} x  RH_{predict_2}"""
print(print_s)

The predict stratum EBT_SAs uses following model:
AGBD = -104.9654541015625 + 6.802174091339111 x RH_50 + 3.9553122520446777 x  RH_98


Also, read the predictor and response offsets from the L4A file.

In [6]:
predictor_offset = hf_l4a['BEAM0110']['agbd_prediction'].attrs['predictor_offset']
response_offset = hf_l4a['BEAM0110']['agbd_prediction'].attrs['response_offset']
# print
print_s = f"""The predictor offset is {predictor_offset}, 
and the response offset is {response_offset}"""
print(print_s)

The predictor offset is 100, 
and the response offset is 0


We will use the predictor offset later when transforming the variables. 

## L2A file and RH metrics
Now let's open the L2A file and read the RH metrics.

In [7]:
# GEDI L2a file
l2a = 'GEDI02_A_2020207182449_O09168_03_T03028_02_003_01_V002.h5'
l2af = path.join('full_orbits', l2a)
hf_l2a = h5py.File(l2af, 'r')

rh_50 = hf_l2a['BEAM0110']['rh'][158560:158561,50][0] # rh_50
rh_98 = hf_l2a['BEAM0110']['rh'][158560:158561,98][0] # rh_98
rh_units = hf_l2a['BEAM0110']['rh'].attrs['units']
shot_number_l2a = hf_l2a['BEAM0110']['shot_number'][158560:158561][0]
selected_algorithm_l2a = hf_l2a['BEAM0110']['selected_algorithm'][158560:158561][0]

print_s = f"""For GEDI shot number {shot_number_l2a}, 
L2A rh_50 is {rh_50} {rh_units}.
L2A rh_98 is {rh_98} {rh_units}.
Selected algorithm group is {selected_algorithm_l2a}."""
print(print_s)

For GEDI shot number 91680600300633870, 
L2A rh_50 is 19.149999618530273 m.
L2A rh_98 is 37.150001525878906 m.
Selected algorithm group is 2.


## xvar
We can now compute predictor variables `xvar` from the L2A RH metrics and compare with the L4A xvar values.

In [8]:
xvar_1_l2a = np.sqrt(rh_50 + predictor_offset) # sqrt x transformation
xvar_2_l2a = np.sqrt(rh_98 + predictor_offset) # sqrt x transformation
xvar_1 = hf_l4a['BEAM0110']['xvar'][158560:158561,0][0]
xvar_2 = hf_l4a['BEAM0110']['xvar'][158560:158561,1][0]

print_s = f"""For GEDI shot number {shot_number_l2a},
L2A derived xvars: {xvar_1_l2a}, {xvar_2_l2a}.
L4A xvars: {xvar_1}, {xvar_2} """
print(print_s)

For GEDI shot number 91680600300633870,
L2A derived xvars: 10.91558517068738, 11.711105905331012.
L4A xvars: 10.9155855178833, 11.711106300354004 


## agbd_t
Let's use the L2A derived xvars from above to compute AGBD in the transformed space and compare the transformed AGBD with that of L4A.

In [9]:
agbd_t_l4a = hf_l4a['BEAM0110']['agbd_t'][158560:158561][0] # l4a agbd_t
agbd_t_l2a = intercept + (coeff_1 * xvar_1_l2a) + (coeff_2 *  xvar_2_l2a) # l2a agbd_t

print_s = f"""For GEDI shot number {shot_number_l2a},
L2A derived agbd_t: {agbd_t_l2a}.
L4A agbd_t: {agbd_t_l4a} """
print(print_s)

For GEDI shot number 91680600300633870,
L2A derived agbd_t: 15.605337210641139.
L4A agbd_t: 15.605335235595703 


## agbd
Now, let's back-transform L2A derived `agbd_t` and compare with the AGBD from L4A.

In [10]:
agbd_l4a = hf_l4a['BEAM0110']['agbd'][158560:158561][0] # l4a agbd
agbd_units = hf_l4a['BEAM0110']['agbd'].attrs['units'] 
bias_correction_value = model_stratum_l4a['bias_correction_value'] # bias_correction_value
agbd_l2a = (agbd_t_l2a**2) * bias_correction_value # snowdon model with sqrt y transform

print_s = f"""For GEDI shot number {shot_number_l2a},
L2A derived estimated AGBD is {agbd_l2a} {agbd_units}.
L4A estimated AGBD {agbd_l4a} {agbd_units}."""
print(print_s)

For GEDI shot number 91680600300633870,
L2A derived estimated AGBD is 271.13409507246865 Mg / ha.
L4A estimated AGBD 271.134033203125 Mg / ha.
